In [1]:
!pip install gmaps
import pandas as pd
import numpy as np
import requests
import json
import gmaps

from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
file="state.csv"

df=pd.read_csv(file)
df

,State,amount
0,CA,318
1,NY,85
2,MA,59
3,TX,24


In [3]:
#Add lst and lgn columns
df["Lat"]=""
df["Lng"]=""
df

,State,amount,Lat,Lng
0,CA,318,,
1,NY,85,,
2,MA,59,,
3,TX,24,,


In [4]:
#Rename the States
df['State'] = df["State"].replace({'CA': 'California'})
df['State'] = df['State'].replace({'NY': 'New York'})
df['State'] = df['State'].replace({'MA': 'Massachusetts'})
df['State'] = df['State'].replace({'TX': 'Texas'})
df['State'] = df['State'].replace({'WA': 'Washington'})

In [5]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in df.iterrows():
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    state = row['State']
    
    # update address key value
    params['address'] = f"{state}"
    
    # make request
    df_lat_lng = requests.get(url, params=params)
    
    # convert to json
    df_lat_lng = df_lat_lng.json()
    
    df.loc[index, "Lat"] = df_lat_lng["results"][0]["geometry"]["location"]["lat"]
    df.loc[index, "Lng"] = df_lat_lng["results"][0]["geometry"]["location"]["lng"]
    
# Visualize to confirm lat lng appear
df.head()

,State,amount,Lat,Lng
0,California,318,36.7783,-119.418
1,New York,85,40.7128,-74.006
2,Massachusetts,59,42.4072,-71.3824
3,Texas,24,31.9686,-99.9018


In [6]:
#Get the coordinates to use in gmaps
coordinates = df[['Lat', 'Lng']]
coordinates

,Lat,Lng
0,36.7783,-119.418
1,40.7128,-74.006
2,42.4072,-71.3824
3,31.9686,-99.9018


In [7]:
fig = gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))